In [ ]:
library(dplyr)
library(class)
library(tidyverse)
library(pROC)
library(glmnet)

#install.packages('glmnetUtils')
library(glmnetUtils)
install.packages("kernlab")
library(kernlab)
library(gbm)
library(caret)

In [ ]:
#lasso_pred <- readRDS("../../data/ATAC_predictions_train_XX_predict_XX/ATAC_pred_lasso_normalized_trimmed_formatted_standardized.rds")
pred <- readRDS(snakemake@input[["input_predictions"]])

In [ ]:
#sample_types <- read.table("../../data/sample_types.txt", header = F, sep = " ")
sample_types <- readRDS(file = snakemake@input[["input_sample_types"]])

colnames(sample_types) <- c("sample", "sample_type")
head(sample_types)

In [ ]:
data <- merge(pred, sample_types, by="sample")
data <- data %>% mutate(sample_type01 = ifelse(sample_type == "Healthy", "Healthy", "Cancer"))
head(data)

In [ ]:
data <- data %>% select(-sample_type)

In [ ]:
data <- data %>% select(-sample)
head(data)

In [ ]:
data$sample_type01 <- as.factor(data$sample_type01)

In [ ]:
cross_validation <- function(dataset, k_inner_cv, k_outer_cv){
    
    observed  <- dataset$sample_type01
   
    return_tibble <- tibble(observed = observed)
    
    for (i in 1:k_outer_cv){ # repeated Cross-validation loop
    
    set.seed(i)
    cvfolds <- cut(seq_len(nrow(dataset)), breaks = k_inner_cv, labels = F)
    cvfolds <- sample(cvfolds)

    predicted <- rep(NA, nrow(dataset))
    
        for (n in 1:k_inner_cv){
            rows      <- which(cvfolds==n)
            testdata  <- dataset[rows,]
            testlabels <- testdata$sample_type01
            testdata <- testdata %>% select(-sample_type01)
            
            traindata <- dataset
         
            traindata <- dataset[-rows,]
           
            trainlabels <- traindata$sample_type01
            #traindata <- traindata %>% select(-sample_type01)

 
            ################# Nested cross validation #######################
            set.seed(0)
            seeds <- vector(mode = "list", length = 11)
            for(i in 1:10) seeds[[i]]<- sample.int(n=1000, 20)
            #for the last model
            seeds[[11]]<-sample.int(1000, 1)

            trControl_svm <- trainControl(method = "repeatedcv", 
                                      seeds = seeds,
                                      number = 10, 
                                      repeats = 1, 
                                      classProbs = TRUE,
                                      verboseIter = ifelse(is.null(getOption('knitr.in.progress')), TRUE, FALSE))

            #svmGrid <- expand.grid(C = c(0.01, 0.1, 0.5, 1, 2, 3, 4, 5, 6, 7))

            fit <- train(sample_type01 ~ .,
                         data = traindata, 
                         method = "svmRadial",
                         tuneLength = 10,
                         trControl = trControl_svm,
                         preProc = c("center", "scale"),
                         verbose=F)
            
            message(plot(fit))
            message("besttune")
            message(fit$bestTune)
            message("fit")
            message(fit)
            #################################################################
            #message(fit)

            fitControl <- trainControl(classProbs = TRUE)
            fit <- train(sample_type01 ~ .,
                         data = traindata,
                         method =  "svmRadial",
                         trControl = fitControl,
                         verbose = FALSE,
                         tuneGrid = data.frame(C = fit$bestTune$C, 
                                               sigma = fit$bestTune$sigma),
                        preProc = c("center", "scale"))
            predicted[rows] <- predict(fit, newdata = testdata, type = "prob")[,2]
        
    }
    current_round_tibble <- tibble(predicted = predicted)
    return_tibble <- cbind(return_tibble, current_round_tibble)
    } # end of outer cv loop
    
    return(return_tibble)
}

In [ ]:
results <- cross_validation(data, k_inner_cv = 10, k_outer_cv = 10)

In [ ]:
results <- tibble(results, .name_repair = "unique")
head(results)

In [ ]:
saveRDS(results, file = snakemake@output[["SVM_output"]])

In [ ]:
# Create multiple curves to plot
rocs <- roc(observed ~ predicted...2 + 
                    predicted...3 + 
                    predicted...4 + 
                    predicted...5 + 
                    predicted...6 +
                    predicted...7 +
                    predicted...8 + 
                    predicted...9 + 
                    predicted...10 + 
                    predicted...11, data = results)
ggroc(rocs)

In [ ]:
roc1 <- roc(results$observed, results$predicted...2)
roc2 <- roc(results$observed, results$predicted...3)
roc3 <- roc(results$observed, results$predicted...4)
roc4 <- roc(results$observed, results$predicted...5)
roc5 <- roc(results$observed, results$predicted...6)
roc6 <- roc(results$observed, results$predicted...7)
roc7 <- roc(results$observed, results$predicted...8)
roc8 <- roc(results$observed, results$predicted...9)
roc9 <- roc(results$observed, results$predicted...10)
roc10 <- roc(results$observed, results$predicted...11)

In [ ]:
paste("Mean AUC over 10 repetitions of 10-fold CV: ", mean(c(roc1$auc, roc2$auc, roc3$auc, roc4$auc, roc5$auc, roc6$auc, roc7$auc, roc8$auc, roc9$auc, roc10$auc)), sep = "")

In [ ]:
results_01 <- results %>% mutate(predicted...2_01 = ifelse(prob...2 > 0.5, 1, 0), 
                                 predicted...3_01 = ifelse(prob...3 > 0.5, 1, 0), 
                                 predicted...4_01 = ifelse(prob...4 > 0.5, 1, 0),
                                 predicted...5_01 = ifelse(prob...5 > 0.5, 1, 0),
                                 predicted...6_01 = ifelse(prob...6 > 0.5, 1, 0),
                                 predicted...7_01 = ifelse(prob...7 > 0.5, 1, 0),
                                 predicted...8_01 = ifelse(prob...8 > 0.5, 1, 0),
                                 predicted...9_01 = ifelse(prob...9 > 0.5, 1, 0),
                                 predicted...10_01 = ifelse(prob...10 > 0.5, 1, 0),
                                 predicted...11_01 = ifelse(prob...11 > 0.5, 1, 0))

In [ ]:
error_rates <- c(mean(results_01$observed != results_01$predicted...2_01), 
                mean(results_01$observed != results_01$predicted...3_01),
               mean(results_01$observed != results_01$predicted...4_01),
               mean(results_01$observed != results_01$predicted...5_01),
               mean(results_01$observed != results_01$predicted...6_01),
               mean(results_01$observed != results_01$predicted...7_01),
               mean(results_01$observed != results_01$predicted...8_01),
               mean(results_01$observed != results_01$predicted...9_01), 
               mean(results_01$observed != results_01$predicted...10_01), 
               mean(results_01$observed != results_01$predicted...11_01))

error_rates

paste("Mean error rate over 10 repetitions of 10-fold CV: ", mean(error_rates), sep = "")

In [ ]:
accuracies <- c(mean(results_01$observed == results_01$predicted...2_01),
               mean(results_01$observed == results_01$predicted...3_01), 
               mean(results_01$observed == results_01$predicted...4_01),
               mean(results_01$observed == results_01$predicted...5_01),
               mean(results_01$observed == results_01$predicted...6_01),
               mean(results_01$observed == results_01$predicted...7_01),
               mean(results_01$observed == results_01$predicted...8_01),
               mean(results_01$observed == results_01$predicted...9_01), 
               mean(results_01$observed == results_01$predicted...10_01), 
               mean(results_01$observed == results_01$predicted...11_01))

accuracies
paste("Mean accuracy over 10 repetitions of 10-fold CV: ", mean(accuracies), sep = "")